In [1]:
import pandas as pd
import os

In [ ]:
#All candidate money data location
cand_money_loc = 'data/all_cand_data/'
# column names
cand_money_cols = ['Cand_Id','Link_Image', 'Cand_Name', 'Cand_Office', 'Cand_Office_St',
       'Cand_Office_Dist', 'Cand_Party_Affiliation',
       'Cand_Incumbent_Challenger_Open_Seat', 'Total_Receipt',
       'Total_Disbursement', 'Cash_On_Hand_COP', 'Debt_Owed_By_Committee',
       'Coverage_End_Date', 'Cand_Street_1', 'Cand_Street_2', 'Cand_City',
       'Cand_State', 'Cand_Zip', 'Individual_Itemized_Contribution',
       'Individual_Unitemized_Contribution', 'Individual_Contribution',
       'Other_Committee_Contribution', 'Party_Committee_Contribution',
       'Cand_Contribution', 'Total_Contribution',
       'Transfer_From_Other_Auth_Committee', 'Cand_Loan', 'Other_Loan',
       'Total_Loan', 'Offsets_To_Operating_Expenditure',
       'Offsets_To_Fundraising', 'Offsets_To_Leagal_Accounting',
       'Other_Receipts', 'Operating_Expenditure',
       'Exempt_Legal_Accounting_Disbursement', 'Fundraising_Disbursement',
       'Transfer_To_Other_Auth_Committee', 'Cand_Loan_Repayment',
       'Other_Loan_Repayment', 'Total_Loan_Repayment', 'Individual_Refund',
       'Party_Committee_Refund', 'Other_Committee_Refund',
       'Total_Contribution_Refund', 'Other_Disbursements', 'Net_Contribution',
       'Net_Operating_Expenditure', 'Cash_On_Hand_BOP',
       'Debt_Owe_To_Committee', 'Coverage_Start_Date']
# year to file dictionary
cand_money_yr2file = {
    2008 : 'candidate_summary_2008.csv',
    2010 : 'candidate_summary_2010.csv',
    2012 : 'candidate_summary_2012.csv',
    2014 : 'candidate_summary_2014.csv',
    2016 : 'candidate_summary_2016.csv',
    2018 : 'candidate_summary_2018.csv',
    2020 : 'candidate_summary_2020.csv',
    2022 : 'candidate_summary_2022.csv',
    2024 : 'candidate_summary_2024.csv'
}

# Finanal data to drop from money dataframe
cols2remove = ['Cand_Street_1',
'Cand_Street_2',
'Cand_City',
'Cand_Zip',
'Cand_Loan',
'Transfer_From_Other_Auth_Committee',
'Total_Loan',
'Offsets_To_Fundraising',
'Offsets_To_Leagal_Accounting',
'Exempt_Legal_Accounting_Disbursement',
'Transfer_To_Other_Auth_Committee',
'Cand_Loan_Repayment',
'Other_Loan_Repayment',
'Total_Loan_Repayment',
'Debt_Owe_To_Committee',
'Coverage_Start_Date',
'Coverage_End_Date',
'Link_Image',
'Individual_Refund',
'Party_Committee_Refund',
'Other_Committee_Refund',
'Cand_Loan_Repayment',
'Other_Loan_Repayment',
'Total_Loan_Repayment',
'Cand_Loan',
'Other_Loan',
'Total_Loan',
'Debt_Owed_By_Committee',
'Debt_Owe_To_Committee',
'Total_Receipt',
'Other_Receipts',
'Transfer_To_Other_Auth_Committee',
'Transfer_From_Other_Auth_Committee',
'Offsets_To_Operating_Expenditure',
'Offsets_To_Fundraising',
'Offsets_To_Leagal_Accounting'
]


### Clean data

In [ ]:
# take df and adjust dollar columns for inflation of the year of the election
def adjust_for_inflation(df):
    """
    Adjust monetary columns in the DataFrame for inflation to 2024 levels.
    
    Args:
        df (pd.DataFrame): DataFrame containing monetary columns and 'CAND_ELECTION_YR'
        
    Returns:
        pd.DataFrame: DataFrame with inflation-adjusted monetary columns
    """
    # Inflation adjustment factors based on election year. pulled for FED website
    inflation_factor = {
    2008: 1.50,
    2010: 1.46,
    2012: 1.39,
    2014: 1.35,
    2016: 1.33,
    2018: 1.27,
    2020: 1.22,
    2022: 1.12,
    2024: 1.0
}

    # List of columns that contain monetary values
    money_columns = [
        'Total_Receipt', 'Total_Disbursement', 'Cash_On_Hand_COP',
        'Debt_Owed_By_Committee', 'Individual_Itemized_Contribution',
        'Individual_Unitemized_Contribution', 'Individual_Contribution',
        'Other_Committee_Contribution', 'Party_Committee_Contribution',
        'Cand_Contribution', 'Total_Contribution',
        'Transfer_From_Other_Auth_Committee', 'Cand_Loan', 'Other_Loan',
        'Total_Loan', 'Offsets_To_Operating_Expenditure',
        'Offsets_To_Fundraising', 'Offsets_To_Leagal_Accounting',
        'Other_Receipts', 'Operating_Expenditure',
        'Exempt_Legal_Accounting_Disbursement', 'Fundraising_Disbursement',
        'Transfer_To_Other_Auth_Committee', 'Cand_Loan_Repayment',
        'Other_Loan_Repayment', 'Total_Loan_Repayment', 'Individual_Refund',
        'Party_Committee_Refund', 'Other_Committee_Refund',
        'Total_Contribution_Refund', 'Other_Disbursements', 'Net_Contribution',
        'Net_Operating_Expenditure', 'Cash_On_Hand_BOP',
        'Debt_Owe_To_Committee'
    ]  
    # Create a copy of the DataFrame to avoid modifying the original
    adjusted_df = df.copy()
    
    # Apply inflation adjustment to each monetary column
    for column in money_columns:    
        # Apply inflation factor based on election year
        for year, factor in inflation_factor.items():
            mask = adjusted_df['Cand_Election_Yr'] == year
            adjusted_df.loc[mask, column] = adjusted_df.loc[mask, column] * factor
    
    return adjusted_df

# append all cand spending file into one DataFrame
def create_omni_accounts_df():
    """
    Create a DataFrame with all candidate accounts.
    """
    global cand_money_yr2file, cand_money_cols
    omni_accounts = pd.DataFrame(columns=cand_money_cols + ['Cand_Election_Yr'])
    
    for year in cand_money_yr2file.keys():
        df = pd.read_csv(os.path.join('data/all_cand_data/', cand_money_yr2file[year]), index_col=0)
        df['Cand_Election_Yr'] = year
        if df is not None:
            omni_accounts = pd.concat([omni_accounts, df], ignore_index=True)
    
    # Create a dictionary to map party abbreviations to full names of the most popular parties
    party_dict = {
        'DEM': 'Democrat',
        'REP': 'Republican',
        'LIB': 'Libertarian',
        'IND': 'Independent',
    }
    # add new column with party names and grouping smaller parties into 'Other'
    omni_accounts.loc[:, ('party')] = omni_accounts['Cand_Party_Affiliation'].map(party_dict)
    omni_accounts.loc[:, ('party')] = omni_accounts['party'].fillna('Other')

    return adjust_for_inflation(omni_accounts)

# create dataframe and save to file for the dashboard to use.
omni_account_df = create_omni_accounts_df()
omni_account_df
omni_account_df.drop(columns=cols2remove, inplace=True)
omni_account_df.to_excel('data/omni_account_df_practice.xlsx')


C:\Users\Josiah\AppData\Local\Temp\ipykernel_13148\1540365081.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  omni_accounts = pd.concat([omni_accounts, df], ignore_index=True)
